In [1]:
import os
import pandas as pd
from pathlib import Path

# Retrieve total consumption for residential
elec_residential = 14481  # PJ UNDP statistics
elec_residential = elec_residential / (3.6e-6) #conversion en MWh
print(f"Electricity consumption : {elec_residential:,.0f} MWh") # MWh
print(f"Electricity consumption : {elec_residential/10**6:,.0f} TWh") # MWh

Electricity consumption : 4,022,500,000 MWh
Electricity consumption : 4,022 TWh


In [2]:
#Energy consumption from the MTF survey (kWh/HH)
if 'ROOT_DIR' not in locals():
  ROOT_DIR = '..'
## MTF survey
mtf_path = ROOT_DIR + "/Residential/Data/MTF survey"
mtf_name = 'MTFData_ListC23.csv'

In [3]:
# Load data into Pandas DataFrame

df_mtfdata = pd.read_csv(mtf_path + "/" + mtf_name)
colMTF = "C23"
df_mtfdata = df_mtfdata[df_mtfdata[colMTF]<14080]# ignore line with very high value (doesn't match the other HH)
df_mtfdata

,HouseholdID,PROVINCE,DISTRICT,TOWNSHIP_COMPOUND_VILLAGE,LOCALITY,EANUM,LOF,ENUMID,SUPID,DOV,...,C180_1,C180_2,C180_3,C180_4,C180_5,U4_1,U4_2,U4_3,U4_4,U4_5
0,1037205,Central,1,chibombo(mwashiala),Rural,372,Nyanja,511,507,1,...,NaN,NaN,NaN,NaN,NaN,Not a member,NaN,NaN,NaN,NaN
1,1112803,Central,2,Railways,Urban,1128,Bemba,507,507,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1112809,Central,2,Railways,Urban,1128,English,507,507,1,...,NaN,NaN,NaN,NaN,NaN,Not a member,NaN,NaN,NaN,NaN
3,1112812,Central,2,railways,Urban,1128,Bemba,511,507,1,...,NaN,NaN,NaN,NaN,NaN,Not a member,NaN,NaN,NaN,NaN
4,1128611,Central,2,Natuseko,Urban,1286,Bemba,507,507,1,...,NaN,NaN,NaN,NaN,NaN,Income generating activities,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,9254803,Southern,9,NEW TOWN SHIP,Urban,2548,Tonga,513,501,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
225,9254806,Southern,9,bbunta,Urban,2548,Tonga,513,501,1,...,NaN,NaN,NaN,NaN,NaN,Religious related activities,NaN,NaN,NaN,NaN
226,9254809,Southern,9,bbunta,Urban,2548,Tonga,501,513,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
227,9254814,Southern,9,bbunta,Urban,2548,Tonga,505,501,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
provinces = df_mtfdata["PROVINCE"].unique()
provinces

array(['Central', 'Copperbelt', 'Eastern', 'Luapula', 'Lusaka',
       'North western', 'Southern', 'WESTERN'], dtype=object)

In [5]:
# Results for all HH: consumption in kWh per month
energycons_res_mtf={}
for province in provinces:
    nb = (df_mtfdata['PROVINCE'] == province).sum()
    min_ = df_mtfdata[(df_mtfdata['PROVINCE'] == province)][colMTF].min()
    max_ = df_mtfdata[(df_mtfdata['PROVINCE'] == province)][colMTF].max()
    mean_ = df_mtfdata[(df_mtfdata['PROVINCE'] == province)][colMTF].mean()
    median_ = df_mtfdata[(df_mtfdata['PROVINCE'] == province)][colMTF].median()
    energycons_res_mtf[province] = {"nb": nb, "min": min_, "max": max_, "mean": mean_, "median": median_}
nb = len(df_mtfdata.index)
min_ = df_mtfdata[colMTF].min()
max_ = df_mtfdata[colMTF].max()
mean_ = df_mtfdata[colMTF].mean()
median_ = df_mtfdata[colMTF].median()
energycons_res_mtf["Total"] = {"nb": nb, "min": min_, "max": max_, "mean": mean_, "median": median_}
energycons_res_mtf = pd.DataFrame(energycons_res_mtf).transpose()
pd.options.display.float_format = '{:,.0f}'.format
energycons_res_mtf
energycons_res_mtf.index = energycons_res_mtf.index.str.upper()

# Results for rural and urban HH kWh per month
energycons_res_mtf_loc={}
locs = df_mtfdata["LOCALITY"].unique()
for loc in locs:
    energycons_res_mtf_tmp={}
    for province in provinces:
        criteria = (df_mtfdata['PROVINCE'] == province) & (df_mtfdata["LOCALITY"] == loc)
        nb = criteria.sum()
        min_ = df_mtfdata[criteria][colMTF].min()
        max_ = df_mtfdata[criteria][colMTF].max()
        mean_ = df_mtfdata[criteria][colMTF].mean()
        median_ = df_mtfdata[criteria][colMTF].median()
        energycons_res_mtf_tmp[province] = {"nb": nb, "min": min_, "max": max_, "mean": mean_, "median": median_}
    energycons_res_mtf_loc[loc] = pd.DataFrame(energycons_res_mtf_tmp).transpose()
pd.options.display.float_format = '{:,.0f}'.format
energycons_res_mtf_loc["Urban"].index = energycons_res_mtf_loc["Urban"].index.str.upper()
energycons_res_mtf_loc["Rural"].index = energycons_res_mtf_loc["Rural"].index.str.upper()



In [6]:
energycons_res_mtf_loc["Rural"]

,nb,min,max,mean,median
CENTRAL,2,150,183,166,166
COPPERBELT,2,200,250,225,225
EASTERN,1,100,100,100,100
LUAPULA,6,208,376,256,242
LUSAKA,5,100,"1,000",280,100
NORTH WESTERN,0,nan,nan,nan,nan
SOUTHERN,7,40,600,215,60
WESTERN,0,nan,nan,nan,nan


In [7]:
energycons_res_mtf_loc["Urban"]

,nb,min,max,mean,median
CENTRAL,4,46,200,112,100
COPPERBELT,91,15,"1,117",258,220
EASTERN,9,100,150,106,100
LUAPULA,6,105,235,182,206
LUSAKA,68,44,750,246,200
NORTH WESTERN,3,44,256,181,243
SOUTHERN,23,50,350,124,100
WESTERN,1,24,24,24,24


In [8]:
energycons_res_mtf

,nb,min,max,mean,median
CENTRAL,6,46,200,130,150
COPPERBELT,93,15,"1,117",258,220
EASTERN,10,100,150,105,100
LUAPULA,12,105,376,219,218
LUSAKA,73,44,"1,000",248,200
NORTH WESTERN,3,44,256,181,243
SOUTHERN,30,40,600,145,100
WESTERN,1,24,24,24,24
TOTAL,228,15,"1,117",226,194


In [9]:
# Results annual kWh per year
energycons_res_mtf["annual"] = energycons_res_mtf["mean"] * 12
energycons_res_mtf["annual"]

CENTRAL         1,558
COPPERBELT      3,093
EASTERN         1,260
LUAPULA         2,630
LUSAKA          2,982
NORTH WESTERN   2,172
SOUTHERN        1,742
WESTERN           288
TOTAL           2,710
Name: annual, dtype: float64